# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
!pip install -q ultralytics kaggle pillow


In [ ]:
import os
import json

kaggle_token = {
    "username": "",  
    "key": ""             
}

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(kaggle_token, f)
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)


In [ ]:
!kaggle datasets download -d fatihkgg/affectnet-yolo-format



In [ ]:
import zipfile

with zipfile.ZipFile("affectnet-yolo-format.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [155]:
import os
import shutil
import random
from collections import defaultdict

def subset_by_class_shuffled(
    src_img_dir: str,
    src_lbl_dir: str,
    dst_img_dir: str,
    dst_lbl_dir: str,
    limit_per_class: int,
    class_count: int = 8
):
   
    available = defaultdict(int)
    for lbl in os.listdir(src_lbl_dir):
        if not lbl.endswith(".txt"):
            continue
        with open(os.path.join(src_lbl_dir, lbl)) as f:
            cls_ids = {int(line.split()[0]) for line in f if line.strip()}
        for cid in cls_ids:
            if 0 <= cid < class_count:
                available[cid] += 1

    shortages = [(cid, available.get(cid, 0)) for cid in range(class_count)
                 if available.get(cid, 0) < limit_per_class]
    if shortages:
        msgs = [f" Class {cid}: {got} available, needs {limit_per_class}"
                for cid, got in shortages]
        raise ValueError("Cannot create subset; shortages detected:\n" + "\n".join(msgs))

    for folder in (dst_img_dir, dst_lbl_dir):
        if os.path.exists(folder):
            shutil.rmtree(folder)
        os.makedirs(folder, exist_ok=True)

    selected_per_class = defaultdict(list)
    for lbl in sorted(os.listdir(src_lbl_dir)):
        if not lbl.endswith(".txt"):
            continue
        with open(os.path.join(src_lbl_dir, lbl)) as f:
            cls_ids = {int(line.split()[0]) for line in f if line.strip()}
        for cid in cls_ids:
            if len(selected_per_class[cid]) < limit_per_class:
                selected_per_class[cid].append(lbl)
                break

    all_selected = []
    for cid, lbls in selected_per_class.items():
        all_selected += [(lbl, cid) for lbl in lbls]
    random.shuffle(all_selected)

    for lbl, cid in all_selected:
        base = os.path.splitext(lbl)[0]
        img_src = None
        for ext in (".jpg", ".png"):
            candidate = os.path.join(src_img_dir, base + ext)
            if os.path.exists(candidate):
                img_src = candidate
                break
        if img_src is None:
            print(f"Skipping {lbl}: image not found")
            continue

        shutil.copy(img_src, os.path.join(dst_img_dir, os.path.basename(img_src)))
        shutil.copy(os.path.join(src_lbl_dir, lbl), os.path.join(dst_lbl_dir, lbl))

    print(f"Subset created: {limit_per_class} images per class, shuffled")
    counts = defaultdict(int)
    for _, cid in all_selected:
        counts[cid] += 1
    for cid in range(class_count):
        print(f" Class {cid}: {counts[cid]} images")


In [156]:
subset_by_class_shuffled(
    src_img_dir   = "data/YOLO_format/train/images",
    src_lbl_dir   = "data/YOLO_format/train/labels",
    dst_img_dir   = "data/YOLO_subset/train/images",
    dst_lbl_dir   = "data/YOLO_subset/train/labels",
    limit_per_class = 800,
    class_count   = 8
)

subset_by_class_shuffled(
    src_img_dir   = "data/YOLO_format/valid/images",
    src_lbl_dir   = "data/YOLO_format/valid/labels",
    dst_img_dir   = "data/YOLO_subset/valid/images",
    dst_lbl_dir   = "data/YOLO_subset/valid/labels",
    limit_per_class = 200,
    class_count   = 8
)


Subset created: 800 images per class, shuffled
 Class 0: 800 images
 Class 1: 800 images
 Class 2: 800 images
 Class 3: 800 images
 Class 4: 800 images
 Class 5: 800 images
 Class 6: 800 images
 Class 7: 800 images
Subset created: 200 images per class, shuffled
 Class 0: 200 images
 Class 1: 200 images
 Class 2: 200 images
 Class 3: 200 images
 Class 4: 200 images
 Class 5: 200 images
 Class 6: 200 images
 Class 7: 200 images


In [104]:
yaml_content = """
train: D:/Sisteme inteligente/scientificProject1/data/YOLO_subset/train/images
val: D:/Sisteme inteligente/scientificProject1/data/YOLO_subset/valid/images
test: D:/Sisteme inteligente/scientificProject1/data/YOLO_subset/test/images

nc: 8
names:
  0: anger
  1: contempt
  2: disgust
  3: fear
  4: happy
  5: neutral
  6: sad
  7: surprise
"""

with open("data/YOLO_subset/data.yaml", "w") as f:
    f.write(yaml_content)


In [106]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.train(data='data/YOLO_subset/data.yaml', epochs=150, batch = 16, imgsz = 96, patience = 30)


New https://pypi.org/project/ultralytics/8.3.144 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.143  Python-3.10.0 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-11370H 3.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/YOLO_subset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=96, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train16, nbs=64, nms=False, opset=None,

train: Scanning D:\Sisteme inteligente\scientificProject1\data\YOLO_subset\train\labels.cache... 6400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6400/6400 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 105.523.1 MB/s, size: 20.2 KB)



val: Scanning D:\Sisteme inteligente\scientificProject1\data\YOLO_subset\valid\labels.cache... 1600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1600/1600 [00:00<?, ?it/s]

Plotting labels to runs\detect\train16\labels.jpg... 


...

    150/150         0G    0.05371     0.8113     0.8989         16         96: 100%|██████████| 400/400 [01:07<00:00,  5.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:08<00:00,  5.61it/s]


                   all       1600       1600      0.602      0.698      0.696      0.696

150 epochs completed in 3.266 hours.
Optimizer stripped from runs\detect\train16\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train16\weights\best.pt, 6.2MB

Validating runs\detect\train16\weights\best.pt...
Ultralytics 8.3.143  Python-3.10.0 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-11370H 3.30GHz)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:07<00:00,  6.27it/s]


                   all       1600       1600      0.604      0.695      0.696      0.696
                 anger        200        200      0.566        0.7      0.682      0.682
              contempt        200        200      0.619      0.755      0.754      0.754
               disgust        200        200      0.504      0.645      0.546      0.546
                  fear        200        200      0.855      0.625      0.814      0.814
                 happy        200        200      0.693      0.855      0.897      0.897
               neutral        200        200      0.483      0.744      0.643      0.643
                   sad        200        200      0.502      0.423      0.458      0.458
              surprise        200        200      0.609      0.815      0.772      0.772
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train16


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001D4B46BEE90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [107]:
from ultralytics import YOLO

model1 = YOLO('yolov8m.pt')

model1.train(
    data='data/YOLO_subset/data.yaml',
    epochs=200,           
    batch=8,              
    imgsz=96,            
    patience=50,          
    name='exp_augmented_m',
    augment=True,
    mosaic=0.5,
    mixup=0.2,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    flipud=0.2,
    fliplr=0.5,
)


100%|██████████| 49.7M/49.7M [00:01<00:00, 43.1MB/s]


Ultralytics 8.3.143  Python-3.10.0 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-11370H 3.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/YOLO_subset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=96, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=exp_augmented_m, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=True, p

train: Scanning D:\Sisteme inteligente\scientificProject1\data\YOLO_subset\train\labels.cache... 6400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6400/6400 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 104.827.4 MB/s, size: 20.2 KB)



val: Scanning D:\Sisteme inteligente\scientificProject1\data\YOLO_subset\valid\labels.cache... 1600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1600/1600 [00:00<?, ?it/s]

...

                   all       1600       1600      0.634      0.753      0.749      0.748



200 epochs completed in 16.960 hours.
Optimizer stripped from runs\detect\exp_augmented_m\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\exp_augmented_m\weights\best.pt, 52.0MB

Validating runs\detect\exp_augmented_m\weights\best.pt...
Ultralytics 8.3.143  Python-3.10.0 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-11370H 3.30GHz)
Model summary (fused): 92 layers, 25,844,392 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 100/100 [01:09<00:00,  1.43it/s]


                   all       1600       1600      0.654      0.769      0.738      0.738
                 anger        200        200      0.626       0.81      0.712      0.712
              contempt        200        200      0.659      0.795      0.738      0.738
               disgust        200        200      0.585      0.715      0.656      0.656
                  fear        200        200      0.795       0.74      0.796      0.796
                 happy        200        200      0.806       0.88      0.926      0.926
               neutral        200        200      0.542      0.825       0.71       0.71
                   sad        200        200      0.591       0.58      0.578      0.578
              surprise        200        200      0.628      0.809       0.79       0.79
Speed: 0.1ms preprocess, 40.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\exp_augmented_m


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001D4C6383490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [108]:
model_export = YOLO("runs/detect/exp_augmented_m/weights/best.pt")
model_export.export(format="onnx")


Ultralytics 8.3.143  Python-3.10.0 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-11370H 3.30GHz)
Model summary (fused): 92 layers, 25,844,392 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from 'runs\detect\exp_augmented_m\weights\best.pt' with input shape (1, 3, 96, 96) BCHW and output shape(s) (1, 12, 189) (49.6 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.53...
ONNX: export success  4.0s, saved as 'runs\detect\exp_augmented_m\weights\best.onnx' (98.7 MB)

Export complete (5.3s)
Results saved to D:\Sisteme inteligente\scientificProject1\runs\detect\exp_augmented_m\weights
Predict:         yolo predict task=detect model=runs\detect\exp_augmented_m\weights\best.onnx imgsz=96  
Validate:        yolo val task=detect model=runs\detect\exp_augmented_m\weights\best.onnx imgsz=96 data=data/YOLO_subset/data.yaml  
Visualize:       https://netron.app


'runs\\detect\\exp_augmented_m\\weights\\best.onnx'

In [153]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

model2 = YOLO(r"D:\Sisteme inteligente\scientificProject1\models\best.onnx")

image_path = r"D:\Sisteme inteligente\scientificProject1\fear.jpg"


results = model2.predict(source=image_path, save=False)
result = results[0]

image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError("Image not found or couldn't be loaded.")

scale = 4
image = cv2.resize(image, (image.shape[1] * scale, image.shape[0] * scale))

boxes = result.boxes.xyxy.cpu().numpy()
scores = result.boxes.conf.cpu().numpy()
class_ids = result.boxes.cls.cpu().numpy().astype(int)
names = result.names

for box, score, class_id in zip(boxes, scores, class_ids):
    x1, y1, x2, y2 = map(int, box * scale)
    label = f"{names[class_id]}: {score:.2f}"

    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, label, (x1, max(20, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 255, 0), 2, lineType=cv2.LINE_AA)

cv2.imshow("YOLOv8 Prediction (Scaled)", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("prediction_output1.png", image)


WARNING Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading D:\Sisteme inteligente\scientificProject1\models\best.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

image 1/1 D:\Sisteme inteligente\scientificProject1\fear.jpg: 96x96 1 fear, 16.1ms
Speed: 0.4ms preprocess, 16.1ms inference, 6.8ms postprocess per image at shape (1, 3, 96, 96)


True

In [138]:
import cv2
from ultralytics import YOLO

model = YOLO(r"D:\Sisteme inteligente\scientificProject1\models\best.onnx")

input_path  = r"D:\Sisteme inteligente\scientificProject1\video.mp4"
output_path = r"D:\Sisteme inteligente\scientificProject1\output_video.mp4"

cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video {input_path}")

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps    = cap.get(cv2.CAP_PROP_FPS)
w      = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
scale  = 1 
out    = cv2.VideoWriter(output_path, fourcc, fps, (w*scale, h*scale))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img96 = cv2.resize(frame, (96, 96), interpolation=cv2.INTER_LINEAR)

    results = model.predict(source=img96, imgsz=96, save=False)
    r = results[0]

    img_in = img96.copy()

    img_out = cv2.resize(frame, (w*scale, h*scale), interpolation=cv2.INTER_LINEAR)


    for box, conf, cls in zip(
        r.boxes.xyxy.cpu().numpy(),
        r.boxes.conf.cpu().numpy(),
        r.boxes.cls.cpu().numpy().astype(int)
    ):
        x1, y1, x2, y2 = box
        x1 = int(x1 / 96 * w * scale)
        y1 = int(y1 / 96 * h * scale)
        x2 = int(x2 / 96 * w * scale)
        y2 = int(y2 / 96 * h * scale)

        label = f"{r.names[cls]} {conf:.2f}"
        cv2.rectangle(img_out, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(img_out, label, (x1, max(20, y1-10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    cv2.imshow("YOLOv8 Video", img_out)
    out.write(img_out)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print("Finished processing, saved to", output_path)


WARNING Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading D:\Sisteme inteligente\scientificProject1\models\best.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

0: 96x96 2 fears, 128.5ms
Speed: 52.6ms preprocess, 128.5ms inference, 178.9ms postprocess per image at shape (1, 3, 96, 96)

0: 96x96 2 fears, 1 sad, 35.5ms
Speed: 3.2ms preprocess, 35.5ms inference, 11.1ms postprocess per image at shape (1, 3, 96, 96)

0: 96x96 1 fear, 1 sad, 28.0ms
Speed: 1.7ms preprocess, 28.0ms inference, 11.9ms postprocess per image at shape (1, 3, 96, 96)

0: 96x96 2 fears, 23.6ms
Speed: 1.0ms preprocess, 23.6ms inference, 2.0ms postprocess per image at shape (1, 3, 96, 96)

0: 96x96 2 fears, 17.1ms
Speed: 0.4ms preprocess, 17.1ms inference, 4.8ms postprocess per image at shape (1, 3, 96, 96)

0: 96x96 2 fears, 20.2ms
Speed: 0.9ms preprocess, 20.2ms inferenc